Plan of Action:
    1. Use starter pack to generate questions to see how good it is at doing it.
    2. If it is bad at it, we will come back and write our own question generator in the future, but we will use it for now.
    3. Start the report generation task
    4. Sample ideas include: Using NLP models to "average out" the repetitive questions for less API usage. Have predetermined categories for types of questions. Each category must be answered in the report. Some examples of category include but are not limited to, author bias, fact cross checking, author reputation, general question asking related to the topic at hand.

This block of code reads the 2024 question document and filters out any questions not rated as '4'. Prints the output to a file.

In [18]:
import pandas as pd
import numpy as np
# Specify the path to the text file
file_path = '2024-question-assessment.txt'

array = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.split("\t")
        if (line[5] == "4" or line[5] == "3"):
            array.append(line)
        
dataFrame = pd.DataFrame(array)
dataFrame.columns = ["AID", "Article", "RID", "Model", "Order", "Assessment", "Useless", "Question"]
dataFrame.drop(columns=["AID", "Model", "RID", "Useless", "Order"], inplace=True)
dataFrame.head()


,Article,Assessment,Question
0,clueweb22-en0030-87-05450,3,What are the qualifications of the creators of...
1,clueweb22-en0030-87-05450,3,What is Anne Lise Marstrand-Jørgensen's expert...
2,clueweb22-en0030-87-05450,3,Does the Danish public service broadcaster DR ...
3,clueweb22-en0030-87-05450,3,What evidence supports the claim that the show...
4,clueweb22-en0030-87-05450,3,What is the broader context of Danish children...


In [ ]:
import pandas as pd
import json 

# Specify the path to the text file
file_path = 'trec-2024-lateral-reading-task2-baseline-documents.json'

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.split("\t")
        print(line)
           

